In [45]:
from gensim import *
import spacy
import nltk
import gensim.downloader #pra trazer um modelo pre-pronto do google notícias pra trabalhar.

In [46]:
for model_name in list(gensim.downloader.info()["models"].keys()):
    print(model_name)

#vamos usar o modelo word2vec-google-news-300

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [ ]:
#chamando o gensim pra fazer o download do modelo certo
g_news = gensim.downloader.load("word2vec-google-news-300")

In [47]:
#aqui vamos procurar pela segunda palavra que mais se adequa a rome, que seria italy.
#Ele nao achou, mas talvez fosse melhor colocar mais dados pra ele entender, ou letra maiuscula etc
capital = g_news.most_similar(["paris", "rome"], ["france"])
print(capital)

[('joel', 0.48578500747680664), ('dubai', 0.4609808027744293), ('oliver', 0.45785653591156006), ('2Bdrm_suite', 0.45590725541114807), ('heidi', 0.4545367360115051), ('samuel', 0.4514700174331665), ('jh', 0.4510999917984009), ('florence', 0.44964927434921265), ('lohan', 0.44291403889656067), ('jessie', 0.44002825021743774)]


In [85]:
#nesse caso estamos usando o word2vec pra ele achar as 4 palavras mais próximas de porsche
w = g_news.most_similar("Porsche", topn=4)
print(w)

[('marcus', 0.6085894703865051), ('brandon', 0.6059526205062866), ('jason', 0.6021084785461426), ('jeff', 0.60173100233078)]


In [49]:
# numa escala de -1 a 1 ele acha o quanto essas palavras sao similares entre si. 0,55 é razoalvelmente alta
c = g_news.similarity("Italy", "Rome")
print(c)

0.5555642


In [50]:
#Agora vamos importar mais bibliotecas e fazer um processamento de dados
import re
import pandas as pd
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from nltk.stem.porter import *

In [51]:
df = pd.read_csv("movies.csv")

In [52]:
#Pegando uma amostra de 20% pra nao ficar muito grande
#Depois, resetando seu índice, pois a amostra bagunça

movies_sample = df.sample(frac=0.2, replace=False).reset_index(drop=True)

In [53]:
movies_sample

,review,sentiment
0,Out to Sea was a great movie. I expected comed...,positive
1,Why is it that a woman cannot be a strong char...,negative
2,Homeward Bound is a beautiful film. Y'know the...,positive
3,"This is a small film , few characters ,theatri...",positive
4,This film has not been seen by me in quite a f...,positive
...,...,...
9995,"OK, what to say about Actium Maximus...<br /><...",negative
9996,Jolene (Heather Graham) operates a night club ...,positive
9997,It was an excellent piece to the puppet series...,positive
9998,I don't know how anyone could hate this movie....,positive


In [54]:
#Vou chamar as stopwords na minha lista sw e depois aplicarei a stematização (que seria cortar a palavra)
#lembrando que eh a lematização que eu abrevio a palavra, mas a stematização eu corto pra um radical fixo
sw_english = set(stopwords.words("english"))
stemmer = PorterStemmer()

In [57]:
def preprocess(string):
    #Deixa apenas elementos alfanumericos
    string = re.sub(r"[^a-zA-Z0-9]+", " ", string)

    #deixa as palavras minusculas
    string = string.lower()

    words = word_tokenize(string)

    #removendo stopwords
    filtered_words = [word for word in words if word not in sw_english]

    #aplicando stematização
    stem_words = [stemmer.stem(w) for w in filtered_words]

    return stem_words

In [58]:
#vou criar uma coluna no meu df chamada filtered_words cujo conteúdo será o texto original
#após aplicação da função acima

movies_sample["filtered_words"] = movies_sample["review"].apply(lambda x: preprocess(x))

In [59]:
#vou jogar o resultado numa lista
words_list = movies_sample["filtered_words"].to_list()

In [60]:
from gensim.models.phrases import Phraser, Phrases, ENGLISH_CONNECTOR_WORDS

In [62]:
#Os n_grams, no caso bi_gram, pois sao duas, pra analisar o contexto das palavras, afinal, word2vec pega a semântica das palavras, nao
#só a frequencia, como com o tf-ifd. Então, como usarei bigrama, estarei analisando a relação entre 2 palavras, o contexto só entre elas duas.

#entao, eu vou passar a lista criada acima, as conexões terão que ser com palavras cuja frequencia mínima seja 1, utilizando os
#conectores, pois as vezes, tirar o ON, IN, OF etc entre 2 palavras pode tirar o sentido.
phrases = Phrases(words_list, min_count=1, connector_words=ENGLISH_CONNECTOR_WORDS, threshold=1)

bigram = Phraser(phrases)

sentences = list(bigram[words_list])

In [64]:
#peguei uma sentença aqui pra poder analisar o que aconteceu
print(sentences[3705])

['feel_good', 'movi_noth', 'great_fun', 'watch', 'sure', 'skim', 'polit_issu', 'sure', 'tri_make', 'good_night', 'good_luck', 'let_tri', 'make', 'anyth_els', 'light_fare', 'br_br', 'enjoy', 'br_br', 'rememb', '1984', 'like', 'becom', 'sophist', 'accord', 'media', 'far', 'watch', 'tend_differ', 'point', 'goldi', 'knew', 'fun', 'fluff', 'went_ahead', 'br_br', 'like', 'lightest', 'fare', 'protocol', 'overboard', 'housesitt', 'wildcat', 'privat_benjamin', 'seem_like', 'old_time', 'foul', 'play', 'death', 'becom', 'first_wive', 'club', 'remak', 'towner', 'goldi_know', 'play', 'everi_role', 'camp', 'get', 'goldi_know', 'realli_well', 'know', 'realli_well', 'br_br', 'alway', 'made_laugh', 'cheer', 'innoc', 'love', 'laugh_everi', 'thing_ever', 'never_tri', 'anyth_els', 'bubbl', 'giggli', 'girl_next', 'door', 'happen', 'pretti_smile', 'laugh', 'alway', 'endear', 'remind', 'life', 'pretti_short', 'got', 'lighten_know', 'old', 'wrinkl', 'suffer', 'one', 'life', 'inevit', 'ailment', 'even_come', '

In [65]:
#A partir disso, a gente chama o word2vec, passa a lista de palavras acima, cujo minimo de aparições seja 2,
# rodando em 2 cpu diferente pra ficar mais rapido
model = Word2Vec(words_list, min_count=2, workers=2)

In [84]:
#Vou aplicar aquele most_similar la de cima pra palavra bad, que me trará as principais palavras mais similares ao bad
model.wv.most_similar("bad")

[('aw', 0.8556991815567017),
 ('suck', 0.8493819236755371),
 ('terribl', 0.8444787859916687),
 ('horribl', 0.8284021615982056),
 ('ok', 0.8104258179664612),
 ('dumb', 0.78822261095047),
 ('lame', 0.7741866707801819),
 ('pathet', 0.771503210067749),
 ('stupid', 0.7707765698432922),
 ('damn', 0.7640008926391602)]

In [ ]:
#Acredito que o professor tenha parado aqui pois já criou um modelo similar ao g_news, pra buscar similaridade entre as palavras.